In [ ]:
import pandas as pd

max_length= 128 

file_path = '../../datasets/task05-methylation/GSM6637962_CpG_coverage20_GRCh38.bed.gz'     
df = pd.read_csv(file_path, sep='\t')
df['ROWID'] = df.index


df['CHROM'] = df['CHROM'].str.replace('chr', '', regex=False)
df=df[~df['CHROM'].str.contains('KI',na=False)]
df=df[~df['CHROM'].str.contains('GL',na=False)]
df=df[~df['CHROM'].str.contains('M',na=False)]


df['SIZE']=max_length

df['START']= df['FROM']-round(max_length / 2)
df['END']= df['START'] + df['SIZE']

df = df.rename(columns={'Percentage':'y'})
df = df.drop(['FROM','TO','Coverage'], axis=1)

df_origin = df
df_origin

In [ ]:
from Bio import SeqIO

sequences = list(SeqIO.parse("/blue/xiaofan/chenyuanhan/data/genome.hg38rg.fa", "fasta"))

def extract_sequence_segment(seq_id, start, end, sequences):
    for seq_record in sequences:
        if seq_record.id == seq_id:
            segment = str(seq_record.seq[start:end])
#            print(segment)
            return segment
    return None
    
rows = []
for index, row in df_origin.iterrows():
    chrom=str(row['CHROM'])
    start=int(row['START'])
    end=int(row['END'])
    y=row['y']
    rowid=row['ROWID']
    segment = extract_sequence_segment(chrom, start, end, sequences)
    rows.append([segment,  y])  # rowid,

columns=['sequence','y']  # 'rowid',
df_sequence = pd.DataFrame(rows, columns=columns)
df_sequence

### Load tokenizer and model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Load pre-trained Caduceus model and tokenizer
model_name = "kuleshov-group/caduceus-ph_seqlen-131k_d_model-256_n_layer-16"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained(model_name, trust_remote_code=True)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

### compute embeddings in batches 

In [ ]:
import dask.dataframe as dd
import pandas as pd
import jax.numpy as jnp
from dask.diagnostics import ProgressBar

# Vectorized tokenization function
def vectorized_tokenizer(subsequences):
    # Tokenize the batch of sequences
    tokens = tokenizer(subsequences, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Move tokens to GPU
    tokens = {key: val.to(device) for key, val in tokens.items()}
    return tokens

# Vectorized embedding function
def vectorized_embedding(tokens):
    # Forward pass to compute last layer embeddings for the batch
    with torch.no_grad():
        outputs = model(**tokens, output_hidden_states=True)  # Enable output of hidden states
        hidden_states = outputs.hidden_states  # Access all hidden states
        last_layer_embeddings = hidden_states[-1]  # Get the last layer embeddings (batch_size, seq_len, hidden_size)
    
    # Compute the mean of the last layer embeddings across the token (sequence) dimension for each sequence in the batch
    # Dimension 1 corresponds to the token/sequence length, so we compute the mean along this axis
    mean_embeddings = torch.mean(last_layer_embeddings, dim=1)  # (batch_size, hidden_size)
    
    # If needed, squeeze out any extra dimensions (though this shouldn't be necessary after mean calculation)
    mean_embeddings_squeezed = mean_embeddings.squeeze(dim=1)

    return mean_embeddings_squeezed

# Tokenization and embedding combined in a batch-wise function
def process_batch(subsequences):
    tokens = vectorized_tokenizer(subsequences)
    embeddings = vectorized_embedding(tokens)
    return embeddings


def apply_get_embeddings_dask(df):
    subsequences = df['sequence'].tolist() 
    embeddings = process_batch(subsequences)  # Process in a vectorized manner
    embeddings_cpu = embeddings.cpu().numpy()
    
    df2 = pd.DataFrame(embeddings_cpu, columns=[f'{i+1}' for i in range(embeddings_cpu.shape[1])])
    df = pd.concat([df.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)
    return df

In [ ]:
%%time

import numpy as np
import pandas as pd
import dask.dataframe as dd
from datasets import load_dataset, load_from_disk

%run preprocess_utility.py

df = df_sequence

typename="methylation"

for chunkid in range(0,1):
    
    #================df -> df's chunks================
    # Assume df is your huge DataFrame
    chunk_size = 10000  # Define the number of rows per chunk
    num_parallel = 10
    
    num_chunks = int(np.ceil(len(df) / chunk_size))  # Calculate the number of chunks
    
    # Split the DataFrame into chunks using array_split
    chunks = np.array_split(df, num_chunks)
    
    # Initialize an empty list to store the processed chunks
    processed_chunks = []
    
    #================process each chunk with dask's ddf================
    # Iterate over each chunk
    for chunk in chunks:
    
        ddf = dd.from_pandas(chunk, npartitions=num_parallel)  # Adjust 'npartitions' based on resources
    
        num_embedding_columns = 256
        
        # Define metadata for the new DataFrame structure
        # Drop the 'embedding' column since it no longer exists
        meta = chunk.copy()
        meta = meta.drop(columns=['embedding'], errors='ignore')  # Drop 'embedding' if it exists
        for i in range(num_embedding_columns):
            meta[f'{i+1}'] = float  # Adjust type as necessary, float is common for embeddings


        # Apply the function in parallel using Dask
        ddf = ddf.map_partitions(apply_get_embeddings_dask, meta=meta)
    
        # Compute the result with progress tracking
        with ProgressBar():
            processed_chunk = ddf.compute()
    
        # Append processed chunk to list
        processed_chunks.append(processed_chunk)

        
    # Concatenate all processed chunks into a final DataFrame
    final_df = pd.concat(processed_chunks, ignore_index=True)

    final_df = final_df.drop(columns=['sequence'])
    final_df = swapfirst2last(final_df)
    
    final_df.to_csv(typename+'_caduceus_embedding_'+str(chunkid)+'.csv', index=False)
    print(f"{typename}_caduceus_embedding_{chunkid}.csv is created.")

final_df